In [10]:
import os

class Parameters:
    def __init__(self):
        self.base_dir = ''
        self.dir_pos_examples = os.path.join(self.base_dir, 'exemplePozitive')
        self.dir_neg_examples = os.path.join(self.base_dir, 'exempleNegative')
        self.dir_test_examples = ("C:/Users/rares/Downloads/testare/testare/")  #teste
        self.path_annotations = ("") #anotari teste
        self.sol_path="rezultate/"
        
        self.dir_save_files = os.path.join(self.base_dir, 'salveazaFisiere')
        if not os.path.exists(self.dir_save_files):
            os.makedirs(self.dir_save_files)
            print('directory created: {} '.format(self.dir_save_files))
        else:
            print('directory {} exists '.format(self.dir_save_files))

        # set the parameters
        self.dim_window_height=48
        self.dim_window_width=48
        self.dim_hog_cell = 12  # dimensiunea celulei
        self.dim_descriptor_cell = 36  # dimensiunea descriptorului unei celule
        self.overlap = 0.3
        self.number_positive_examples = 6977  # numarul exemplelor pozitive
        self.number_negative_examples = 18000  # numarul exemplelor negative
        self.overlap = 0.3
        self.has_annotations = True
        self.threshold = 0

In [11]:
import numpy as np
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import glob
import cv2 as cv
import pdb
import pickle
import ntpath
from copy import deepcopy
import timeit
import os
from skimage.feature import hog


class FacialDetector:
    def __init__(self, params:Parameters):
        self.params = params
        self.best_model = None

        
        
    def rescale_bbox(self,bbox, from_scale, to_scale):
        rescaled_bbox = [int(coord * to_scale / from_scale) for coord in bbox]
        return rescaled_bbox
    
    def resize_to_specific_size(self,image, target_width, target_height):
        resized_image = cv.resize(image, (target_width, target_height))

        return resized_image
    
    
    def rescale_image(self,image, scale_factor):
        new_width = int(image.shape[1] * scale_factor)
        new_height = int(image.shape[0] * scale_factor)

        rescaled_image = cv.resize(image, (new_width, new_height))

        return rescaled_image




    def intersection_over_union(self, bbox_a, bbox_b):
        x_a = max(bbox_a[0], bbox_b[0])
        y_a = max(bbox_a[1], bbox_b[1])
        x_b = min(bbox_a[2], bbox_b[2])
        y_b = min(bbox_a[3], bbox_b[3])

        inter_area = max(0, x_b - x_a + 1) * max(0, y_b - y_a + 1)

        box_a_area = (bbox_a[2] - bbox_a[0] + 1) * (bbox_a[3] - bbox_a[1] + 1)
        box_b_area = (bbox_b[2] - bbox_b[0] + 1) * (bbox_b[3] - bbox_b[1] + 1)

        iou = inter_area / float(box_a_area + box_b_area - inter_area)

        return iou
    


    def train_classifier(self, training_examples, train_labels,person=None):
        svm_file_name = os.path.join(self.params.dir_save_files, 'best_model_%d_%d_%d' %
                                     (self.params.dim_hog_cell, self.params.number_negative_examples,
                                      self.params.number_positive_examples))
        if person:
            svm_file_name+="_"+person
        if os.path.exists(svm_file_name):
            self.best_model = pickle.load(open(svm_file_name, 'rb'))
            return

        best_accuracy = 0
        best_c = 0
        best_model = None
        Cs = [10 ** -5, 10 ** -4,  10 ** -3,  10 ** -2, 10 ** -1,1]
        for c in Cs:
            print('Antrenam un clasificator pentru c=%f' % c)
            model = LinearSVC(C=c)
            model.fit(training_examples, train_labels)
            acc = model.score(training_examples, train_labels)
            print(acc)
            if acc > best_accuracy:
                best_accuracy = acc
                best_c = c
                best_model = deepcopy(model)
                
        
                

        print('Performanta clasificatorului optim pt c = %f' % best_c)
        # salveaza clasificatorul
        pickle.dump(best_model, open(svm_file_name, 'wb'))

        # vizualizeaza cat de bine sunt separate exemplele pozitive de cele negative dupa antrenare
        # ideal ar fi ca exemplele pozitive sa primeasca scoruri > 0, iar exemplele negative sa primeasca scoruri < 0
        scores = best_model.decision_function(training_examples)
        self.best_model = best_model
        positive_scores = scores[train_labels > 0]
        negative_scores = scores[train_labels <= 0]
        


        plt.plot(np.sort(positive_scores))
        plt.plot(np.zeros(len(positive_scores)))
        plt.plot(np.sort(negative_scores))
        plt.xlabel('Nr example antrenare')
        plt.ylabel('Scor clasificator')
        plt.title('Distributia scorurilor clasificatorului pe exemplele de antrenare')
        plt.legend(['Scoruri exemple pozitive', '0', 'Scoruri exemple negative'])
        plt.show()



    def non_maximal_suppression(self, image_detections, image_scores, image_size):
        """
        Detectiile cu scor mare suprima detectiile ce se suprapun cu acestea dar au scor mai mic.
        Detectiile se pot suprapune partial, dar centrul unei detectii nu poate
        fi in interiorul celeilalte detectii.
        :param image_detections:  numpy array de dimensiune NX4, unde N este numarul de detectii.
        :param image_scores: numpy array de dimensiune N
        :param image_size: tuplu, dimensiunea imaginii
        :return: image_detections si image_scores care sunt maximale.
        """

        # xmin, ymin, xmax, ymax
        x_out_of_bounds = np.where(image_detections[:, 2] > image_size[1])[0]
        y_out_of_bounds = np.where(image_detections[:, 3] > image_size[0])[0]
        #print(x_out_of_bounds, y_out_of_bounds)
        image_detections[x_out_of_bounds, 2] = image_size[1]
        image_detections[y_out_of_bounds, 3] = image_size[0]
        sorted_indices = np.flipud(np.argsort(image_scores))
        sorted_image_detections = image_detections[sorted_indices]
        sorted_scores = image_scores[sorted_indices]

        is_maximal = np.ones(len(image_detections)).astype(bool)
        iou_threshold = 0.3
        for i in range(len(sorted_image_detections) - 1):
            if is_maximal[i] == True:  # don't change to 'is True' because is a numpy True and is not a python True :)
                for j in range(i + 1, len(sorted_image_detections)):
                    if is_maximal[j] == True:  # don't change to 'is True' because is a numpy True and is not a python True :)
                        if self.intersection_over_union(sorted_image_detections[i],sorted_image_detections[j]) > iou_threshold:is_maximal[j] = False
                        else:  # verificam daca centrul detectiei este in mijlocul detectiei cu scor mai mare
                            c_x = (sorted_image_detections[j][0] + sorted_image_detections[j][2]) / 2
                            c_y = (sorted_image_detections[j][1] + sorted_image_detections[j][3]) / 2
                            if sorted_image_detections[i][0] <= c_x <= sorted_image_detections[i][2] and \
                                    sorted_image_detections[i][1] <= c_y <= sorted_image_detections[i][3]:
                                is_maximal[j] = False
        return sorted_image_detections[is_maximal], sorted_scores[is_maximal]

    def run(self):
        """
        Aceasta functie returneaza toate detectiile ( = ferestre) pentru toate imaginile din self.params.dir_test_examples
        Directorul cu numele self.params.dir_test_examples contine imagini ce
        pot sau nu contine fete. Aceasta functie ar trebui sa detecteze fete atat pe setul de
        date MIT+CMU dar si pentru alte imagini
        Functia 'non_maximal_suppression' suprimeaza detectii care se suprapun (protocolul de evaluare considera o detectie duplicata ca fiind falsa)
        Suprimarea non-maximelor se realizeaza pe pentru fiecare imagine.
        :return:
        detections: numpy array de dimensiune NX4, unde N este numarul de detectii pentru toate imaginile.
        detections[i, :] = [x_min, y_min, x_max, y_max]
        scores: numpy array de dimensiune N, scorurile pentru toate detectiile pentru toate imaginile.
        file_names: numpy array de dimensiune N, pentru fiecare detectie trebuie sa salvam numele imaginii.
        (doar numele, nu toata calea).
        """
        scales=[1.         ,0.87055,    0.75785828, 0.66069345, 0.57622779, 0.50270779,
 0.43855988, 0.38239679, 0.33287077, 0.28877435, 0.2499512,  0.21516408,
 0.18401213, 0.15607649, 0.13190697]

        test_images_path = os.path.join(self.params.dir_test_examples, '*.jpg')
        #print(test_images_path)
        test_files = glob.glob(test_images_path)
        detections = None  # array cu toate detectiile pe care le obtinem
        scores = np.array([])  # array cu toate scorurile pe care le obtinem
        file_names = np.array([])  # array cu fisiele, in aceasta lista fisierele vor aparea de mai multe ori, pentru fiecare
        # detectie din imagine, numele imaginii va aparea in aceasta lista
        w = self.best_model.coef_.T
        bias = self.best_model.intercept_[0]
        num_test_images = len(test_files)
        descriptors_to_return = []
        for i in range(num_test_images):
            start_time = timeit.default_timer()
            print('Procesam imaginea de testare %d/%d..' % (i, num_test_images))
            img = cv.imread(test_files[i], cv.IMREAD_GRAYSCALE)
            img_c=img
            image_scores = np.array([])
            image_detections = np.array([])
            for scale in scales:
                img=self.rescale_image(img_c,scale)
                hog_descriptors = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                                      cells_per_block=(2, 2), feature_vector=False)
                num_cols = img.shape[1] // self.params.dim_hog_cell -1
                num_rows = img.shape[0] // self.params.dim_hog_cell -1
                num_cell_in_template_height = self.params.dim_window_height // self.params.dim_hog_cell-1
                num_cell_in_template_width= self.params.dim_window_width // self.params.dim_hog_cell-1 
                

                for y in range(0, num_rows - num_cell_in_template_height):
                    for x in range(0, num_cols - num_cell_in_template_width):
                        descr = hog_descriptors[y:y + num_cell_in_template_height, x:x + num_cell_in_template_width].flatten()
                        score = np.dot(descr, w)[0] + bias
                        if score > self.params.threshold:
                            x_min = int(x * self.params.dim_hog_cell)
                            y_min = int(y * self.params.dim_hog_cell)
                            x_max = int(x * self.params.dim_hog_cell + self.params.dim_window)
                            y_max = int(y * self.params.dim_hog_cell + self.params.dim_window)
                            box=[x_min,y_min,x_max,y_max]
                            new_box=self.rescale_bbox(box,scale,1)
                            if image_detections.size>0:
                                image_detections=np.vstack([image_detections,new_box])
                            else:
                                image_detections=np.array([new_box])
                            image_scores=np.append(image_scores,score)
            if len(image_scores) > 0:
                image_detections, image_scores = self.non_maximal_suppression(np.array(image_detections),
                                                                              np.array(image_scores), img_c.shape)
                
                
            if len(image_scores) > 0:
                if detections is None:
                    detections = image_detections
                else:
                    detections = np.concatenate((detections, image_detections))
                scores = np.append(scores, image_scores)
                short_name = ntpath.basename(test_files[i])
                image_names = [short_name for ww in range(len(image_scores))]
                file_names = np.append(file_names, image_names)

            end_time = timeit.default_timer()
            print('Timpul de procesarea al imaginii de testare %d/%d este %f sec.'
                  % (i, num_test_images, end_time - start_time))

        return detections, scores, file_names    

    def get_poz_person_descriptors(self,person):
        positive_descriptors=[]
        path=self.params.dir_pos_examples
        images=os.listdir(path)
        for image in images:
            if image.split(".")[0].split("_")[1]==person:
                full_path=os.path.join(path,image)
                img=cv.imread(full_path,cv.IMREAD_GRAYSCALE)
                features = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                               cells_per_block=(2, 2), feature_vector=True)
                positive_descriptors.append(features)
                
                if self.params.use_flip_images:
                    features = hog(np.fliplr(img), pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                                   cells_per_block=(2, 2), feature_vector=True)
                    positive_descriptors.append(features)
        return np.array(positive_descriptors)
    
    def get_neg_person_descriptors(self,person,negative=[]):
        negative_descriptors=[]
        path=self.params.dir_pos_examples
        images=os.listdir(path)
        i=0
        for image in images:
            if image.split(".")[0].split("_")[1]!=person:
                full_path=os.path.join(path,image)
                img=cv.imread(full_path,cv.IMREAD_GRAYSCALE)
                features = hog(img, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                               cells_per_block=(2, 2), feature_vector=True)
                negative_descriptors.append(features)
                
                if self.params.use_flip_images:
                    features = hog(np.fliplr(img), pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                                   cells_per_block=(2, 2), feature_vector=True)
                    negative_descriptors.append(features)
        negative_descriptors=np.array(negative_descriptors)
        if len(negative)!=0:
            negative_descriptors = np.concatenate((negative_descriptors, negative), axis=0)
        return (negative_descriptors)
    '''
    def train_recognisition(self):
    
    def face_recognition(self,detections):
    '''
        
        
    def run_person(self,detections,file_names):
        """
        Aceasta functie returneaza toate detectiile ( = ferestre) pentru toate imaginile din self.params.dir_test_examples
        Directorul cu numele self.params.dir_test_examples contine imagini ce
        pot sau nu contine fete. Aceasta functie ar trebui sa detecteze fete atat pe setul de
        date MIT+CMU dar si pentru alte imagini
        Functia 'non_maximal_suppression' suprimeaza detectii care se suprapun (protocolul de evaluare considera o detectie duplicata ca fiind falsa)
        Suprimarea non-maximelor se realizeaza pe pentru fiecare imagine.
        :return:
        detections: numpy array de dimensiune NX4, unde N este numarul de detectii pentru toate imaginile.
        detections[i, :] = [x_min, y_min, x_max, y_max]
        scores: numpy array de dimensiune N, scorurile pentru toate detectiile pentru toate imaginile.
        file_names: numpy array de dimensiune N, pentru fiecare detectie trebuie sa salvam numele imaginii.
        (doar numele, nu toata calea).
        """
        scales=[1.         ,0.87055,    0.75785828, 0.66069345, 0.57622779, 0.50270779,
 0.43855988, 0.38239679, 0.33287077, 0.28877435, 0.2499512,  0.21516408,
 0.18401213, 0.15607649, 0.13190697]

        test_images_path = (self.params.dir_test_examples)
        detections_new = None  # array cu toate detectiile pe care le obtinem
        scores = np.array([])  # array cu toate scorurile pe care le obtinem
        file_names_new = np.array([])  # array cu fisiele, in aceasta lista fisierele vor aparea de mai multe ori, pentru fiecare
        # detectie din imagine, numele imaginii va aparea in aceasta lista
        w = self.best_model.coef_.T
        bias = self.best_model.intercept_[0]
        num_test_images = len(detections)
        descriptors_to_return = []
        i=0
        while i < (detections.shape[0]):
            path=os.path.join(test_images_path,file_names[i])
            start_time = timeit.default_timer()
            print('Procesam imaginea de testare %d/%d..' % (i, num_test_images))
            img = cv.imread(path, cv.IMREAD_GRAYSCALE)
            aux=file_names[i]
            while i<detections.shape[0] and file_names[i]==aux:
                image_scores = np.array([])
                image_detections = np.array([])
                x_min, y_min, x_max, y_max=detections[i]
                img_run=img[y_min:y_max,x_min:x_max]
                img_c=img_run
                ok=False
                for scale in scales:
                    if ok:
                        break
                    img_cop=self.rescale_image(img_c,scale)
                    hog_descriptors = hog(img_run, pixels_per_cell=(self.params.dim_hog_cell, self.params.dim_hog_cell),
                                          cells_per_block=(2, 2), feature_vector=False)
                    num_cols = img_run.shape[1] // self.params.dim_hog_cell -1
                    num_rows = img_run.shape[0] // self.params.dim_hog_cell -1
                    num_cell_in_template_height = self.params.dim_window_height // self.params.dim_hog_cell-1
                    num_cell_in_template_width= self.params.dim_window_width // self.params.dim_hog_cell-1 
                    for y in range(0, num_rows - num_cell_in_template_height):
                        if ok:
                            break
                        for x in range(0, num_cols - num_cell_in_template_width):
                            descr = hog_descriptors[y:y + num_cell_in_template_height, x:x + num_cell_in_template_width].flatten()
                            score = np.dot(descr, w)[0] + bias
                            if score > self.params.threshold:
                                '''
                                x_min = int(x * self.params.dim_hog_cell)
                                y_min = int(y * self.params.dim_hog_cell)
                                x_max = int(x * self.params.dim_hog_cell + self.params.dim_window)
                                y_max = int(y * self.params.dim_hog_cell + self.params.dim_window)
                                '''
                                new_box=[x_min,y_min,x_max,y_max]
                                #cv.imshow()
                                ok=True
                                if image_detections.size>0:
                                    image_detections=np.vstack([image_detections,new_box])
                                else:
                                    image_detections=np.array([new_box])
                                image_scores=np.append(image_scores,score)
                                break


                if len(image_scores) > 0:
                    if detections_new is None:
                        detections_new = image_detections
                    else:
                        detections_new = np.concatenate((detections_new, image_detections))
                    scores = np.append(scores, image_scores)
                    short_name = ntpath.basename(file_names[i])
                    image_names = [short_name for ww in range(len(image_scores))]
                    file_names_new = np.append(file_names_new, image_names)
                
                i+=1


                end_time = timeit.default_timer()
                print('Timpul de procesarea al imaginii de testare %d/%d este %f sec.'
                      % (i, num_test_images, end_time - start_time))

        return detections_new, scores, file_names_new
    
    
    


In [12]:
params: Parameters = Parameters()
params.dim_window=36
params.dim_window_width = 36
params.dim_window_height = 36# exemplele pozitive (fete de oameni cropate) au 36x36 pixeli
params.dim_hog_cell = 6  # dimensiunea celulei
params.overlap = 0.3
params.number_positive_examples = 6977  # numarul exemplelor pozitive
params.number_negative_examples = 72283  # numarul exemplelor negative

params.threshold = 0 # toate ferestrele cu scorul > threshold si maxime locale devin detectii
params.has_annotations = True

params.use_hard_mining = False  # (optional)antrenare cu exemple puternic negative
params.use_flip_images = True  # adauga imaginile cu fete oglindite

if params.use_flip_images:
    params.number_positive_examples *= 2

facial_detector: FacialDetector = FacialDetector(params)
#good=facial_detector.get_positive_examples()
#bad=facial_detector.get_negative_examples()
#print(len(good),len(bad))

# Pasii 1+2+3. Incarcam exemplele pozitive (cropate) si exemple negative generate
# verificam daca sunt deja existente
#print(len(np.ones(params.number_positive_examples)), len(np.zeros(negative_features.shape[0])))
facial_detector.train_classifier("", "")

# Pasul 5. (optional) Antrenare cu exemple puternic negative (detectii cu scor >0 din cele 274 de imagini negative)
# Daca implementati acest pas ar trebui sa modificati functia FacialDetector.run()
# astfel incat sa va returneze descriptorii detectiilor cu scor > 0 din cele 274 imagini negative
# completati codul in continuare
# TODO:  (optional)  completeaza codul in continuare

detections, scores, file_names = facial_detector.run()
np.save(os.path.join(params.sol_path, "task1/detections_all_faces.npy"), detections)
np.save(os.path.join(params.sol_path, "task1/scores_all_faces.npy"), scores)
np.save(os.path.join(params.sol_path, "task1/file_names_all_faces.npy"), file_names)

directory salveazaFisiere exists 
Procesam imaginea de testare 0/200..
Timpul de procesarea al imaginii de testare 0/200 este 0.828249 sec.
Procesam imaginea de testare 1/200..
Timpul de procesarea al imaginii de testare 1/200 este 0.640843 sec.
Procesam imaginea de testare 2/200..
Timpul de procesarea al imaginii de testare 2/200 este 0.657567 sec.
Procesam imaginea de testare 3/200..
Timpul de procesarea al imaginii de testare 3/200 este 0.701479 sec.
Procesam imaginea de testare 4/200..
Timpul de procesarea al imaginii de testare 4/200 este 0.730489 sec.
Procesam imaginea de testare 5/200..
Timpul de procesarea al imaginii de testare 5/200 este 0.678369 sec.
Procesam imaginea de testare 6/200..
Timpul de procesarea al imaginii de testare 6/200 este 0.672420 sec.
Procesam imaginea de testare 7/200..
Timpul de procesarea al imaginii de testare 7/200 este 0.752867 sec.
Procesam imaginea de testare 8/200..
Timpul de procesarea al imaginii de testare 8/200 este 0.692222 sec.
Procesam ima

Timpul de procesarea al imaginii de testare 76/200 este 0.685043 sec.
Procesam imaginea de testare 77/200..
Timpul de procesarea al imaginii de testare 77/200 este 0.769475 sec.
Procesam imaginea de testare 78/200..
Timpul de procesarea al imaginii de testare 78/200 este 0.592476 sec.
Procesam imaginea de testare 79/200..
Timpul de procesarea al imaginii de testare 79/200 este 0.618005 sec.
Procesam imaginea de testare 80/200..
Timpul de procesarea al imaginii de testare 80/200 este 0.659629 sec.
Procesam imaginea de testare 81/200..
Timpul de procesarea al imaginii de testare 81/200 este 0.585928 sec.
Procesam imaginea de testare 82/200..
Timpul de procesarea al imaginii de testare 82/200 este 0.670438 sec.
Procesam imaginea de testare 83/200..
Timpul de procesarea al imaginii de testare 83/200 este 0.881821 sec.
Procesam imaginea de testare 84/200..
Timpul de procesarea al imaginii de testare 84/200 este 0.985162 sec.
Procesam imaginea de testare 85/200..
Timpul de procesarea al imag

Timpul de procesarea al imaginii de testare 151/200 este 0.637885 sec.
Procesam imaginea de testare 152/200..
Timpul de procesarea al imaginii de testare 152/200 este 0.525983 sec.
Procesam imaginea de testare 153/200..
Timpul de procesarea al imaginii de testare 153/200 este 0.601184 sec.
Procesam imaginea de testare 154/200..
Timpul de procesarea al imaginii de testare 154/200 este 0.643717 sec.
Procesam imaginea de testare 155/200..
Timpul de procesarea al imaginii de testare 155/200 este 0.659922 sec.
Procesam imaginea de testare 156/200..
Timpul de procesarea al imaginii de testare 156/200 este 0.581033 sec.
Procesam imaginea de testare 157/200..
Timpul de procesarea al imaginii de testare 157/200 este 0.627134 sec.
Procesam imaginea de testare 158/200..
Timpul de procesarea al imaginii de testare 158/200 este 0.726134 sec.
Procesam imaginea de testare 159/200..
Timpul de procesarea al imaginii de testare 159/200 este 0.701248 sec.
Procesam imaginea de testare 160/200..
Timpul de 

In [13]:
def run(person):
    params.dir_pos_examples=os.path.join(params.base_dir, "exemplePozitive")
    if person =="fred":
        params.dim_window_width = 36
        params.dim_window_height = 48
        params.number_positive_examples=2
        params.dim_hog_cell=6
        params.number_negative_examples =  67511
    if person =="barney":
        params.dim_window_width = 36
        params.dim_window_height = 36
        params.number_positive_examples=13954
        params.dim_hog_cell=6
        params.number_negative_examples =  72283
    if person =="wilma":
        params.dim_window_width = 36
        params.dim_window_height = 48
        params.number_positive_examples=1
        params.dim_hog_cell=4
        params.number_negative_examples =  67511
    if person =="betty":
        params.dim_window_width = 36
        params.dim_window_height = 48
        params.number_positive_examples=1
        params.dim_hog_cell=4
        params.number_negative_examples =  67511
        
    params.threshold=0
    facial_detector.train_classifier("", "",person)
    detections2, scores2, file_names2 = facial_detector.run()
    np.save(os.path.join(params.sol_path, f"task2/detections_{person}.npy"), detections2)
    np.save(os.path.join(params.sol_path, f"task2/scores_{person}.npy"), scores2)
    np.save(os.path.join(params.sol_path, f"task2/file_names_{person}.npy"), file_names2)


In [14]:
run("fred")
run("barney")
run("betty")
run("wilma")

Procesam imaginea de testare 0/200..
Timpul de procesarea al imaginii de testare 0/200 este 0.555889 sec.
Procesam imaginea de testare 1/200..
Timpul de procesarea al imaginii de testare 1/200 este 0.587305 sec.
Procesam imaginea de testare 2/200..
Timpul de procesarea al imaginii de testare 2/200 este 0.591459 sec.
Procesam imaginea de testare 3/200..
Timpul de procesarea al imaginii de testare 3/200 este 0.507849 sec.
Procesam imaginea de testare 4/200..
Timpul de procesarea al imaginii de testare 4/200 este 0.559759 sec.
Procesam imaginea de testare 5/200..
Timpul de procesarea al imaginii de testare 5/200 este 0.585441 sec.
Procesam imaginea de testare 6/200..
Timpul de procesarea al imaginii de testare 6/200 este 0.588989 sec.
Procesam imaginea de testare 7/200..
Timpul de procesarea al imaginii de testare 7/200 este 0.592876 sec.
Procesam imaginea de testare 8/200..
Timpul de procesarea al imaginii de testare 8/200 este 0.653149 sec.
Procesam imaginea de testare 9/200..
Timpul de

Timpul de procesarea al imaginii de testare 76/200 este 0.587727 sec.
Procesam imaginea de testare 77/200..
Timpul de procesarea al imaginii de testare 77/200 este 0.565300 sec.
Procesam imaginea de testare 78/200..
Timpul de procesarea al imaginii de testare 78/200 este 0.589945 sec.
Procesam imaginea de testare 79/200..
Timpul de procesarea al imaginii de testare 79/200 este 0.539019 sec.
Procesam imaginea de testare 80/200..
Timpul de procesarea al imaginii de testare 80/200 este 0.578394 sec.
Procesam imaginea de testare 81/200..
Timpul de procesarea al imaginii de testare 81/200 este 0.580237 sec.
Procesam imaginea de testare 82/200..
Timpul de procesarea al imaginii de testare 82/200 este 0.728722 sec.
Procesam imaginea de testare 83/200..
Timpul de procesarea al imaginii de testare 83/200 este 0.600975 sec.
Procesam imaginea de testare 84/200..
Timpul de procesarea al imaginii de testare 84/200 este 0.602426 sec.
Procesam imaginea de testare 85/200..
Timpul de procesarea al imag

Timpul de procesarea al imaginii de testare 151/200 este 1.071058 sec.
Procesam imaginea de testare 152/200..
Timpul de procesarea al imaginii de testare 152/200 este 1.068227 sec.
Procesam imaginea de testare 153/200..
Timpul de procesarea al imaginii de testare 153/200 este 1.092589 sec.
Procesam imaginea de testare 154/200..
Timpul de procesarea al imaginii de testare 154/200 este 1.104061 sec.
Procesam imaginea de testare 155/200..
Timpul de procesarea al imaginii de testare 155/200 este 1.256837 sec.
Procesam imaginea de testare 156/200..
Timpul de procesarea al imaginii de testare 156/200 este 1.135387 sec.
Procesam imaginea de testare 157/200..
Timpul de procesarea al imaginii de testare 157/200 este 1.139206 sec.
Procesam imaginea de testare 158/200..
Timpul de procesarea al imaginii de testare 158/200 este 1.140352 sec.
Procesam imaginea de testare 159/200..
Timpul de procesarea al imaginii de testare 159/200 este 1.137071 sec.
Procesam imaginea de testare 160/200..
Timpul de 

Timpul de procesarea al imaginii de testare 27/200 este 1.112165 sec.
Procesam imaginea de testare 28/200..
Timpul de procesarea al imaginii de testare 28/200 este 1.237717 sec.
Procesam imaginea de testare 29/200..
Timpul de procesarea al imaginii de testare 29/200 este 1.216272 sec.
Procesam imaginea de testare 30/200..
Timpul de procesarea al imaginii de testare 30/200 este 1.162370 sec.
Procesam imaginea de testare 31/200..
Timpul de procesarea al imaginii de testare 31/200 este 1.092743 sec.
Procesam imaginea de testare 32/200..
Timpul de procesarea al imaginii de testare 32/200 este 1.101251 sec.
Procesam imaginea de testare 33/200..
Timpul de procesarea al imaginii de testare 33/200 este 1.036932 sec.
Procesam imaginea de testare 34/200..
Timpul de procesarea al imaginii de testare 34/200 este 1.036640 sec.
Procesam imaginea de testare 35/200..
Timpul de procesarea al imaginii de testare 35/200 este 1.120321 sec.
Procesam imaginea de testare 36/200..
Timpul de procesarea al imag

Timpul de procesarea al imaginii de testare 103/200 este 1.359419 sec.
Procesam imaginea de testare 104/200..
Timpul de procesarea al imaginii de testare 104/200 este 1.333867 sec.
Procesam imaginea de testare 105/200..
Timpul de procesarea al imaginii de testare 105/200 este 1.343863 sec.
Procesam imaginea de testare 106/200..
Timpul de procesarea al imaginii de testare 106/200 este 1.210531 sec.
Procesam imaginea de testare 107/200..
Timpul de procesarea al imaginii de testare 107/200 este 1.296404 sec.
Procesam imaginea de testare 108/200..
Timpul de procesarea al imaginii de testare 108/200 este 1.451352 sec.
Procesam imaginea de testare 109/200..
Timpul de procesarea al imaginii de testare 109/200 este 1.386550 sec.
Procesam imaginea de testare 110/200..
Timpul de procesarea al imaginii de testare 110/200 este 1.532148 sec.
Procesam imaginea de testare 111/200..
Timpul de procesarea al imaginii de testare 111/200 este 1.456564 sec.
Procesam imaginea de testare 112/200..
Timpul de 

...

Timpul de procesarea al imaginii de testare 54/200 este 3.921338 sec.
Procesam imaginea de testare 55/200..
Timpul de procesarea al imaginii de testare 55/200 este 3.827178 sec.
Procesam imaginea de testare 56/200..
Timpul de procesarea al imaginii de testare 56/200 este 3.614734 sec.
Procesam imaginea de testare 57/200..
Timpul de procesarea al imaginii de testare 57/200 este 3.745732 sec.
Procesam imaginea de testare 58/200..
Timpul de procesarea al imaginii de testare 58/200 este 3.683811 sec.
Procesam imaginea de testare 59/200..
Timpul de procesarea al imaginii de testare 59/200 este 3.685453 sec.
Procesam imaginea de testare 60/200..
Timpul de procesarea al imaginii de testare 60/200 este 3.615321 sec.
Procesam imaginea de testare 61/200..
Timpul de procesarea al imaginii de testare 61/200 este 3.367119 sec.
Procesam imaginea de testare 62/200..
Timpul de procesarea al imaginii de testare 62/200 este 3.456451 sec.
Procesam imaginea de testare 63/200..
Timpul de procesarea al imag

Timpul de procesarea al imaginii de testare 130/200 este 1.902545 sec.
Procesam imaginea de testare 131/200..
Timpul de procesarea al imaginii de testare 131/200 este 1.913443 sec.
Procesam imaginea de testare 132/200..
Timpul de procesarea al imaginii de testare 132/200 este 1.922886 sec.
Procesam imaginea de testare 133/200..
Timpul de procesarea al imaginii de testare 133/200 este 1.988880 sec.
Procesam imaginea de testare 134/200..
Timpul de procesarea al imaginii de testare 134/200 este 1.926987 sec.
Procesam imaginea de testare 135/200..
Timpul de procesarea al imaginii de testare 135/200 este 1.978523 sec.
Procesam imaginea de testare 136/200..
Timpul de procesarea al imaginii de testare 136/200 este 1.954845 sec.
Procesam imaginea de testare 137/200..
Timpul de procesarea al imaginii de testare 137/200 este 1.950016 sec.
Procesam imaginea de testare 138/200..
Timpul de procesarea al imaginii de testare 138/200 este 1.982453 sec.
Procesam imaginea de testare 139/200..
Timpul de 

Timpul de procesarea al imaginii de testare 5/200 este 0.817524 sec.
Procesam imaginea de testare 6/200..
Timpul de procesarea al imaginii de testare 6/200 este 0.836136 sec.
Procesam imaginea de testare 7/200..
Timpul de procesarea al imaginii de testare 7/200 este 0.817210 sec.
Procesam imaginea de testare 8/200..
Timpul de procesarea al imaginii de testare 8/200 este 0.845438 sec.
Procesam imaginea de testare 9/200..
Timpul de procesarea al imaginii de testare 9/200 este 0.844617 sec.
Procesam imaginea de testare 10/200..
Timpul de procesarea al imaginii de testare 10/200 este 0.827962 sec.
Procesam imaginea de testare 11/200..
Timpul de procesarea al imaginii de testare 11/200 este 0.872426 sec.
Procesam imaginea de testare 12/200..
Timpul de procesarea al imaginii de testare 12/200 este 0.839182 sec.
Procesam imaginea de testare 13/200..
Timpul de procesarea al imaginii de testare 13/200 este 0.834581 sec.
Procesam imaginea de testare 14/200..
Timpul de procesarea al imaginii de t

Timpul de procesarea al imaginii de testare 81/200 este 0.837285 sec.
Procesam imaginea de testare 82/200..
Timpul de procesarea al imaginii de testare 82/200 este 0.889764 sec.
Procesam imaginea de testare 83/200..
Timpul de procesarea al imaginii de testare 83/200 este 0.839087 sec.
Procesam imaginea de testare 84/200..
Timpul de procesarea al imaginii de testare 84/200 este 0.825979 sec.
Procesam imaginea de testare 85/200..
Timpul de procesarea al imaginii de testare 85/200 este 0.850821 sec.
Procesam imaginea de testare 86/200..
Timpul de procesarea al imaginii de testare 86/200 este 0.810640 sec.
Procesam imaginea de testare 87/200..
Timpul de procesarea al imaginii de testare 87/200 este 0.836299 sec.
Procesam imaginea de testare 88/200..
Timpul de procesarea al imaginii de testare 88/200 este 0.835768 sec.
Procesam imaginea de testare 89/200..
Timpul de procesarea al imaginii de testare 89/200 este 0.840166 sec.
Procesam imaginea de testare 90/200..
Timpul de procesarea al imag

Timpul de procesarea al imaginii de testare 156/200 este 0.866644 sec.
Procesam imaginea de testare 157/200..
Timpul de procesarea al imaginii de testare 157/200 este 0.828078 sec.
Procesam imaginea de testare 158/200..
Timpul de procesarea al imaginii de testare 158/200 este 0.877500 sec.
Procesam imaginea de testare 159/200..
Timpul de procesarea al imaginii de testare 159/200 este 0.811178 sec.
Procesam imaginea de testare 160/200..
Timpul de procesarea al imaginii de testare 160/200 este 0.851602 sec.
Procesam imaginea de testare 161/200..
Timpul de procesarea al imaginii de testare 161/200 este 0.816228 sec.
Procesam imaginea de testare 162/200..
Timpul de procesarea al imaginii de testare 162/200 este 0.813159 sec.
Procesam imaginea de testare 163/200..
Timpul de procesarea al imaginii de testare 163/200 este 0.807639 sec.
Procesam imaginea de testare 164/200..
Timpul de procesarea al imaginii de testare 164/200 este 0.839749 sec.
Procesam imaginea de testare 165/200..
Timpul de 

In [15]:
def yolo_run_2(person):
    net = cv.dnn.readNet("salveazaFisiere/yolov4-flintstone_last_2.weights", "salveazaFisiere/yolov4-flintstone-2.cfg")
    classes = []
    with open("C:/Users/rares/darknet/data/obj.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getUnconnectedOutLayersNames()

    validation_folder = params.dir_test_examples
    detections = None  # array cu toate detectiile pe care le obtinem
    scores = np.array([])  # array cu toate scorurile pe care le obtinem
    file_names = np.array([])
    
    test_images_path = os.path.join(params.dir_test_examples, '*.jpg')
    test_files = glob.glob(test_images_path)
    i=0

    for file in os.listdir(validation_folder):
        start_time = timeit.default_timer()
        image_path = os.path.join(validation_folder, file)
        img = cv.imread(image_path)
        height, width, _ = img.shape
        blob = cv.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(layer_names)
        image_scores = np.array([])
        image_detections = np.array([])

        # Process the output
        for out in outs:
            for detection in out:
                result = detection[5:]
                class_id = np.argmax(result)
                confidence = result[class_id]
                if confidence > 0.0 and classes[class_id]==person:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x_min = max(0, int(center_x - w / 2))
                    y_min = max(0, int(center_y - h / 2))
                    x_max = min(width, int(center_x + w / 2))
                    y_max = min(height, int(center_y + h / 2))

                    box=[x_min,y_min,x_max,y_max]
                    #cv.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    #cv.putText(img, classes[class_id], (x_min, y_min - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    if image_detections.size>0:
                        image_detections=np.vstack([image_detections,box])
                    else:
                        image_detections=np.array([box])
                    image_scores=np.append(image_scores,confidence)
        if len(image_scores) > 0:
            image_detections, image_scores = facial_detector.non_maximal_suppression(np.array(image_detections),
                                                                          np.array(image_scores), img.shape)

        
        if len(image_scores) > 0:
            if detections is None:
                detections = image_detections
            else:
                detections = np.concatenate((detections, image_detections))
            scores = np.append(scores, image_scores)
            short_name = ntpath.basename(test_files[i])
            image_names = [short_name for ww in range(len(image_scores))]
            file_names = np.append(file_names, image_names)
        i+=1
        end_time = timeit.default_timer()
        print('Timpul de procesarea al imaginii de testare %d/%d este %f sec.'
              % (i,200, end_time - start_time))
        
    np.save(os.path.join(params.sol_path, f"bonus_task2/detections_{person}.npy"), detections)
    np.save(os.path.join(params.sol_path, f"bonus_task2/scores_{person}.npy"), scores)
    np.save(os.path.join(params.sol_path, f"bonus_task2/file_names_{person}.npy"), file_names)


    return detections, scores, file_names
                          
                     
yolo_run_2("Fred")
yolo_run_2("Barney")
yolo_run_2("Wilma")
yolo_run_2("Betty")





Timpul de procesarea al imaginii de testare 1/200 este 1.091258 sec.
Timpul de procesarea al imaginii de testare 2/200 este 0.369175 sec.
Timpul de procesarea al imaginii de testare 3/200 este 0.377582 sec.
Timpul de procesarea al imaginii de testare 4/200 este 0.387213 sec.
Timpul de procesarea al imaginii de testare 5/200 este 0.387680 sec.
Timpul de procesarea al imaginii de testare 6/200 este 0.366227 sec.
Timpul de procesarea al imaginii de testare 7/200 este 0.362990 sec.
Timpul de procesarea al imaginii de testare 8/200 este 0.361868 sec.
Timpul de procesarea al imaginii de testare 9/200 este 0.368869 sec.
Timpul de procesarea al imaginii de testare 10/200 este 0.371903 sec.
Timpul de procesarea al imaginii de testare 11/200 este 0.376906 sec.
Timpul de procesarea al imaginii de testare 12/200 este 0.416752 sec.
Timpul de procesarea al imaginii de testare 13/200 este 0.400801 sec.
Timpul de procesarea al imaginii de testare 14/200 este 0.403102 sec.
Timpul de procesarea al imagi

Timpul de procesarea al imaginii de testare 118/200 este 0.581027 sec.
Timpul de procesarea al imaginii de testare 119/200 este 0.491736 sec.
Timpul de procesarea al imaginii de testare 120/200 este 0.581737 sec.
Timpul de procesarea al imaginii de testare 121/200 este 0.503897 sec.
Timpul de procesarea al imaginii de testare 122/200 este 0.542964 sec.
Timpul de procesarea al imaginii de testare 123/200 este 0.489669 sec.
Timpul de procesarea al imaginii de testare 124/200 este 0.595786 sec.
Timpul de procesarea al imaginii de testare 125/200 este 0.471652 sec.
Timpul de procesarea al imaginii de testare 126/200 este 0.468068 sec.
Timpul de procesarea al imaginii de testare 127/200 este 0.499367 sec.
Timpul de procesarea al imaginii de testare 128/200 este 0.541888 sec.
Timpul de procesarea al imaginii de testare 129/200 este 0.484298 sec.
Timpul de procesarea al imaginii de testare 130/200 este 0.489506 sec.
Timpul de procesarea al imaginii de testare 131/200 este 0.473216 sec.
Timpul

Timpul de procesarea al imaginii de testare 34/200 este 0.532905 sec.
Timpul de procesarea al imaginii de testare 35/200 este 0.551129 sec.
Timpul de procesarea al imaginii de testare 36/200 este 0.541715 sec.
Timpul de procesarea al imaginii de testare 37/200 este 0.571031 sec.
Timpul de procesarea al imaginii de testare 38/200 este 0.456890 sec.
Timpul de procesarea al imaginii de testare 39/200 este 0.474549 sec.
Timpul de procesarea al imaginii de testare 40/200 este 0.463902 sec.
Timpul de procesarea al imaginii de testare 41/200 este 0.460863 sec.
Timpul de procesarea al imaginii de testare 42/200 este 0.488886 sec.
Timpul de procesarea al imaginii de testare 43/200 este 0.532034 sec.
Timpul de procesarea al imaginii de testare 44/200 este 0.494462 sec.
Timpul de procesarea al imaginii de testare 45/200 este 0.474234 sec.
Timpul de procesarea al imaginii de testare 46/200 este 0.469748 sec.
Timpul de procesarea al imaginii de testare 47/200 este 0.493469 sec.
Timpul de procesarea

Timpul de procesarea al imaginii de testare 151/200 este 0.517224 sec.
Timpul de procesarea al imaginii de testare 152/200 este 0.473651 sec.
Timpul de procesarea al imaginii de testare 153/200 este 0.471381 sec.
Timpul de procesarea al imaginii de testare 154/200 este 0.467463 sec.
Timpul de procesarea al imaginii de testare 155/200 este 0.473395 sec.
Timpul de procesarea al imaginii de testare 156/200 este 0.477444 sec.
Timpul de procesarea al imaginii de testare 157/200 este 0.490742 sec.
Timpul de procesarea al imaginii de testare 158/200 este 0.464259 sec.
Timpul de procesarea al imaginii de testare 159/200 este 0.485106 sec.
Timpul de procesarea al imaginii de testare 160/200 este 0.485979 sec.
Timpul de procesarea al imaginii de testare 161/200 este 0.469307 sec.
Timpul de procesarea al imaginii de testare 162/200 este 0.557124 sec.
Timpul de procesarea al imaginii de testare 163/200 este 0.498438 sec.
Timpul de procesarea al imaginii de testare 164/200 este 0.478259 sec.
Timpul

Timpul de procesarea al imaginii de testare 68/200 este 0.466698 sec.
Timpul de procesarea al imaginii de testare 69/200 este 0.482047 sec.
Timpul de procesarea al imaginii de testare 70/200 este 0.482623 sec.
Timpul de procesarea al imaginii de testare 71/200 este 0.479567 sec.
Timpul de procesarea al imaginii de testare 72/200 este 0.479985 sec.
Timpul de procesarea al imaginii de testare 73/200 este 0.471079 sec.
Timpul de procesarea al imaginii de testare 74/200 este 0.457251 sec.
Timpul de procesarea al imaginii de testare 75/200 este 0.488151 sec.
Timpul de procesarea al imaginii de testare 76/200 este 0.454916 sec.
Timpul de procesarea al imaginii de testare 77/200 este 0.475590 sec.
Timpul de procesarea al imaginii de testare 78/200 este 0.465885 sec.
Timpul de procesarea al imaginii de testare 79/200 este 0.466201 sec.
Timpul de procesarea al imaginii de testare 80/200 este 0.465175 sec.
Timpul de procesarea al imaginii de testare 81/200 este 0.494629 sec.
Timpul de procesarea

Timpul de procesarea al imaginii de testare 184/200 este 0.487633 sec.
Timpul de procesarea al imaginii de testare 185/200 este 0.464924 sec.
Timpul de procesarea al imaginii de testare 186/200 este 0.462142 sec.
Timpul de procesarea al imaginii de testare 187/200 este 0.464635 sec.
Timpul de procesarea al imaginii de testare 188/200 este 0.473008 sec.
Timpul de procesarea al imaginii de testare 189/200 este 0.494724 sec.
Timpul de procesarea al imaginii de testare 190/200 este 0.461427 sec.
Timpul de procesarea al imaginii de testare 191/200 este 0.458648 sec.
Timpul de procesarea al imaginii de testare 192/200 este 0.473134 sec.
Timpul de procesarea al imaginii de testare 193/200 este 0.461371 sec.
Timpul de procesarea al imaginii de testare 194/200 este 0.462701 sec.
Timpul de procesarea al imaginii de testare 195/200 este 0.488083 sec.
Timpul de procesarea al imaginii de testare 196/200 este 0.459860 sec.
Timpul de procesarea al imaginii de testare 197/200 este 0.465217 sec.
Timpul

Timpul de procesarea al imaginii de testare 101/200 este 0.576643 sec.
Timpul de procesarea al imaginii de testare 102/200 este 0.572539 sec.
Timpul de procesarea al imaginii de testare 103/200 este 0.549148 sec.
Timpul de procesarea al imaginii de testare 104/200 este 0.496797 sec.
Timpul de procesarea al imaginii de testare 105/200 este 0.506190 sec.
Timpul de procesarea al imaginii de testare 106/200 este 0.580656 sec.
Timpul de procesarea al imaginii de testare 107/200 este 0.518054 sec.
Timpul de procesarea al imaginii de testare 108/200 este 0.595423 sec.
Timpul de procesarea al imaginii de testare 109/200 este 0.531205 sec.
Timpul de procesarea al imaginii de testare 110/200 este 0.498834 sec.
Timpul de procesarea al imaginii de testare 111/200 este 0.508149 sec.
Timpul de procesarea al imaginii de testare 112/200 este 0.499673 sec.
Timpul de procesarea al imaginii de testare 113/200 este 0.534390 sec.
Timpul de procesarea al imaginii de testare 114/200 este 0.491831 sec.
Timpul

(array([[107, 103, 205, 209],
        [ 71, 135, 194, 274],
        [ 64, 131, 197, 271],
        [ 94,  97, 256, 278],
        [101, 127, 200, 207],
        [127, 105, 228, 211],
        [132,  98, 242, 221],
        [415, 101, 460, 144],
        [263, 114, 341, 212],
        [ 95, 109, 191, 217],
        [209,  81, 276, 147],
        [130,  76, 178, 138],
        [119, 102, 217, 194],
        [ 79, 132, 124, 191],
        [ 96, 173, 171, 259],
        [ 87, 147, 130, 193],
        [274, 118, 380, 230],
        [155, 103, 255, 188],
        [276,  90, 372, 196],
        [262,  96, 321, 161],
        [225,  90, 333, 194],
        [256,  81, 368, 244],
        [278,  92, 328, 150],
        [ 92, 113, 200, 227],
        [ 73, 128, 185, 220],
        [ 79, 180, 146, 249],
        [114, 108, 207, 217],
        [119, 116, 245, 235],
        [270, 123, 368, 229],
        [264,  81, 356, 188],
        [288, 131, 342, 176],
        [262,  69, 331, 146],
        [245,  77, 397, 206],
        [2

In [16]:
def yolo_run_1():
    net = cv.dnn.readNet("salveazaFisiere/yolov4-flintstone-1_1000.weights", "salveazaFisiere/yolov4-flintstone-1.cfg")
    classes = []
    with open("C:/Users/rares/darknet/data/obj.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getUnconnectedOutLayersNames()

    validation_folder = params.dir_test_examples
    detections = None  # array cu toate detectiile pe care le obtinem
    scores = np.array([])  # array cu toate scorurile pe care le obtinem
    file_names = np.array([])
    
    test_images_path = os.path.join(params.dir_test_examples, '*.jpg')
    test_files = glob.glob(test_images_path)
    i=0

    for file in os.listdir(validation_folder):
        start_time = timeit.default_timer()
        image_path = os.path.join(validation_folder, file)
        img = cv.imread(image_path)
        height, width, _ = img.shape
        blob = cv.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(layer_names)
        image_scores = np.array([])
        image_detections = np.array([])

        # Process the output
        for out in outs:
            for detection in out:
                result = detection[5:]
                class_id = np.argmax(result)
                confidence = result[class_id]
                if confidence > 0.0:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x_min = max(0, int(center_x - w / 2))
                    y_min = max(0, int(center_y - h / 2))
                    x_max = min(width, int(center_x + w / 2))
                    y_max = min(height, int(center_y + h / 2))

                    box=[x_min,y_min,x_max,y_max]
                    #cv.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    #cv.putText(img, classes[class_id], (x_min, y_min - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    if image_detections.size>0:
                        image_detections=np.vstack([image_detections,box])
                    else:
                        image_detections=np.array([box])
                    image_scores=np.append(image_scores,confidence)
        if len(image_scores) > 0:
            image_detections, image_scores = facial_detector.non_maximal_suppression(np.array(image_detections),
                                                                          np.array(image_scores), img.shape)

        
        if len(image_scores) > 0:
            if detections is None:
                detections = image_detections
            else:
                detections = np.concatenate((detections, image_detections))
            scores = np.append(scores, image_scores)
            short_name = ntpath.basename(test_files[i])
            image_names = [short_name for ww in range(len(image_scores))]
            file_names = np.append(file_names, image_names)
        i+=1
        end_time = timeit.default_timer()
        print('Timpul de procesarea al imaginii de testare %d/%d este %f sec.'
              % (i,200, end_time - start_time))


    np.save(os.path.join(params.sol_path, f"bonus_task1/detections_all_faces.npy"), detections)
    np.save(os.path.join(params.sol_path, f"bonus_task1/scores_all_faces.npy"), scores)
    np.save(os.path.join(params.sol_path, f"bonus_task1/file_names_all_faces.npy"), file_names)
    return detections, scores, file_names
                                     
yolo_run_1()

Timpul de procesarea al imaginii de testare 1/200 este 1.155937 sec.
Timpul de procesarea al imaginii de testare 2/200 este 0.565222 sec.
Timpul de procesarea al imaginii de testare 3/200 este 0.584937 sec.
Timpul de procesarea al imaginii de testare 4/200 este 0.588932 sec.
Timpul de procesarea al imaginii de testare 5/200 este 0.494450 sec.
Timpul de procesarea al imaginii de testare 6/200 este 0.541145 sec.
Timpul de procesarea al imaginii de testare 7/200 este 0.489865 sec.
Timpul de procesarea al imaginii de testare 8/200 este 0.527358 sec.
Timpul de procesarea al imaginii de testare 9/200 este 0.557629 sec.
Timpul de procesarea al imaginii de testare 10/200 este 0.521834 sec.
Timpul de procesarea al imaginii de testare 11/200 este 0.528075 sec.
Timpul de procesarea al imaginii de testare 12/200 este 0.544355 sec.
Timpul de procesarea al imaginii de testare 13/200 este 0.545937 sec.
Timpul de procesarea al imaginii de testare 14/200 este 0.580898 sec.
Timpul de procesarea al imagi

Timpul de procesarea al imaginii de testare 118/200 este 0.479471 sec.
Timpul de procesarea al imaginii de testare 119/200 este 0.483933 sec.
Timpul de procesarea al imaginii de testare 120/200 este 0.609927 sec.
Timpul de procesarea al imaginii de testare 121/200 este 0.552767 sec.
Timpul de procesarea al imaginii de testare 122/200 este 0.596789 sec.
Timpul de procesarea al imaginii de testare 123/200 este 0.625686 sec.
Timpul de procesarea al imaginii de testare 124/200 este 0.571525 sec.
Timpul de procesarea al imaginii de testare 125/200 este 0.794258 sec.
Timpul de procesarea al imaginii de testare 126/200 este 0.689109 sec.
Timpul de procesarea al imaginii de testare 127/200 este 0.675951 sec.
Timpul de procesarea al imaginii de testare 128/200 este 0.728973 sec.
Timpul de procesarea al imaginii de testare 129/200 este 0.610069 sec.
Timpul de procesarea al imaginii de testare 130/200 este 0.492575 sec.
Timpul de procesarea al imaginii de testare 131/200 este 0.497313 sec.
Timpul

(array([[103,  88, 225, 268],
        [256, 103, 332, 211],
        [269, 109, 356, 215],
        ...,
        [197, 111, 308, 266],
        [275,  83, 321, 141],
        [155,  74, 205, 140]]),
 array([0.99412203, 0.97303468, 0.99438423, 0.98968416, 0.8297798 ,
        0.99350661, 0.92150372, 0.93963546, 0.99529493, 0.96330553,
        0.98486221, 0.99840796, 0.97956312, 0.94693542, 0.99523127,
        0.98081833, 0.98532397, 0.99823523, 0.99617094, 0.79278249,
        0.99606365, 0.96904534, 0.9968285 , 0.9992404 , 0.99832439,
        0.99500394, 0.99892873, 0.99794263, 0.99893266, 0.93107277,
        0.97966856, 0.97851485, 0.9714669 , 0.96035135, 0.91544628,
        0.98173511, 0.97346067, 0.93148434, 0.99684846, 0.99575651,
        0.33850825, 0.22630788, 0.99025726, 0.97819322, 0.99703211,
        0.95959848, 0.24292853, 0.99498576, 0.94661283, 0.88858813,
        0.98922378, 0.98770916, 0.4337818 , 0.98225778, 0.99714845,
        0.99184829, 0.98919296, 0.98596901, 0.99466908, 0